In [1]:
from human_eval.data import write_jsonl, read_problems

In [2]:
from bia_bob import bob

In [7]:
#from bia_bob._machinery import OLLAMA_BASE_URL
#base_url = OLLAMA_BASE_URL
#model='codellama'
#bob.initialize(endpoint='ollama', model=model)

In [8]:
base_url = None
model='gpt-3.5-turbo-1106'
bob.initialize(model=model)

In [9]:
def generate_one_completion(prompt):
    from bia_bob._utilities import generate_response_from_openai
    
    response = generate_response_from_openai(model=model, system_prompt='', user_prompt=prompt, chat_history=[], base_url=base_url)

    if '[PYTHON]' in response and '[/PYTHON]' in response:
        response = response.replace('[PYTHON]', '```')
        response = response.replace('[/PYTHON]', '```')
        
    if '```' in response:
        response = response.replace('```python', '```')
        
        response = response.split('```')[-2]

    print("Q:", prompt)
    print("A:", response)
    
    return response

In [10]:
problems = read_problems('../data/human-eval-bia.jsonl')

num_samples_per_task = 2
samples = [
    dict(task_id=task_id, completion=generate_one_completion(problems[task_id]["prompt"]))
    for _ in range(num_samples_per_task)
    for task_id in problems
]
write_jsonl(f"samples_{model}.jsonl", samples)

Q: import skimage
import numpy as np

def label_binary_image_and_count_labels(binary_image):
    """
    Consumes as input a binary image, applies connected component labeling to it, 
    counts the labeled objects and returns their count as single number.
    """
A: 
    labeled_image, num_labels = skimage.measure.label(binary_image, background=0, return_num=True)
    return num_labels

Q: import skimage
import numpy as np

def apply_otsu_threshold_and_count_postiive_pixels(image):
    """
    Takes an image, applies Otus's threshold method to it to create a binary image and 
    counts the positive pixels.
    """
A: 
from skimage import filters
import numpy as np

def apply_otsu_threshold_and_count_postiive_pixels(image):
    threshold_value = filters.threshold_otsu(image)
    binary_image = image > threshold_value
    positive_pixels_count = np.sum(binary_image)
    return binary_image, positive_pixels_count

Q: import skimage
import numpy as np

def label_binary_image_and_count_la